In [1]:
import re
from underthesea import sent_tokenize
from khmernltk import sentence_tokenize
from vncorenlp import VnCoreNLP
from transformers import AutoModel, AutoTokenizer

2021-01-05 10:27:07,809 - INFO     - root                      - Loaded model from D:\anaconda\envs\myenv\lib\site-packages\khmernltk\word_tokenize\sklearn_crf_ner_10000.sav
2021-01-05 10:27:07,827 - INFO     - root                      - Loaded model from D:\anaconda\envs\myenv\lib\site-packages\khmernltk\pos_tag\sklearn_crf_pos_alt_0.9849.sav


In [2]:
annotator = VnCoreNLP("C:\\Users\\Dell 7577\\Downloads\\VnCoreNLP-master\\VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m')


2021-01-05 10:27:11,152 - INFO     - vncorenlp.vncorenlp       - Starting server on: http://127.0.0.1:56438
2021-01-05 10:27:11,157 - INFO     - vncorenlp.vncorenlp       - Server ID: 17292
2021-01-05 10:27:11,163 - DEBUG    - urllib3.connectionpool    - Starting new HTTP connection (1): 127.0.0.1:56438
2021-01-05 10:27:12,781 - DEBUG    - urllib3.connectionpool    - http://127.0.0.1:56438 "GET / HTTP/1.1" 200 None
2021-01-05 10:27:12,786 - DEBUG    - urllib3.connectionpool    - Starting new HTTP connection (1): 127.0.0.1:56438
2021-01-05 10:27:12,797 - DEBUG    - urllib3.connectionpool    - http://127.0.0.1:56438 "GET /annotators HTTP/1.1" 200 None
2021-01-05 10:27:12,800 - INFO     - vncorenlp.vncorenlp       - The server is now available on: http://127.0.0.1:56438


In [3]:
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
phobert = AutoModel.from_pretrained("vinai/phobert-base", output_hidden_states = True)

2021-01-05 10:27:15,813 - DEBUG    - urllib3.connectionpool    - Starting new HTTPS connection (1): huggingface.co:443
2021-01-05 10:27:18,754 - DEBUG    - urllib3.connectionpool    - https://huggingface.co:443 "HEAD /vinai/phobert-base/resolve/main/config.json HTTP/1.1" 200 0
2021-01-05 10:27:18,761 - DEBUG    - urllib3.connectionpool    - Starting new HTTPS connection (1): huggingface.co:443
2021-01-05 10:27:19,713 - DEBUG    - urllib3.connectionpool    - https://huggingface.co:443 "HEAD /vinai/phobert-base/resolve/main/vocab.txt HTTP/1.1" 200 0
2021-01-05 10:27:19,719 - DEBUG    - urllib3.connectionpool    - Starting new HTTPS connection (1): huggingface.co:443
2021-01-05 10:27:20,675 - DEBUG    - urllib3.connectionpool    - https://huggingface.co:443 "HEAD /vinai/phobert-base/resolve/main/bpe.codes HTTP/1.1" 200 0
Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.
2021-01-05 10:27:20,820 - DEBUG    - urllib3.connecti

In [60]:
def read_file(file_path, language='vi', sen=False):
    res = []
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        res = [re.sub('\n|\u200b', '', line).strip() for line in lines]
        res = [line for line in res if len(line) > 0]
        if sen:
            return res[0] 
        else: 
            res_segmented = []
            if language == 'vi':
                for line in res:
                    print(line)
                    print(sent_tokenize(line))
                    print('-'*50) 
                    res_segmented += sent_tokenize(line)
            else: 
                for line in res:
                    res_segmented += sentence_tokenize(line)
            # punctuation = '[!"#$%&\'()*+,./;<=>?@[\\]^_`{|}~]'
            # res = [re.sub(punctuation, '', line) for line in res]
            return res_segmented

def preprocess_text(text, language, sen=False):
    if not sen:
        res = []       
        if language == 'vi':
            for line in text: 
                res += sent_tokenize(line)        
        else:
            for line in text:
                
                res += sentence_tokenize(line)
        punctuation = '[!"#$%&\'()*+,./;<=>?@[\\]^_`{|}~]'
        res = [re.sub(punctuation, '', line) for line in res]
        res = [line.lower() for line in res]
        return res
    else: 
        punctuation = '[!"#$%&\'()*+,./;<=>?@[\\]^_`{|}~]'
        text = re.sub(punctuation, '', text)
        return text.lower()



In [12]:
def segment_sentence(sen):
    word_segmented_sen = annotator.tokenize(sen)
    # word_segmented_sen = annotator.annotate(sen)['sentences'][0]
    res = ''
    # for word in word_segmented_sen:
    #     if word['nerLabel'] not in ['B-PER', 'B-LOC', 'B-ORG', 'B-MISC']:
    #         res += word['form'] + ' '
    for sen in word_segmented_sen:
        for word in sen:
            res += word + ' '
    return res.strip() 

In [62]:
# km_doc = read_file('./test_km.txt')
vi_doc = read_file('./vi_50_sens.txt')

Ý đã đánh bại Bồ Đào Nha với tỉ số 31-5 ở Bảng C Giải vô địch Rugby thế giới 2007 tại Parc des Princes, Pari, Pháp.
['Ý đã đánh bại Bồ Đào Nha với tỉ số 31-5 ở Bảng C Giải vô địch Rugby thế giới 2007 tại Parc des Princes, Pari, Pháp.']
--------------------------------------------------
Andrea Maisi đã mở tỉ số cho Ý ở phút thứ tư với một quả try.
['Andrea Maisi đã mở tỉ số cho Ý ở phút thứ tư với một quả try.']
--------------------------------------------------
Chiếm thế áp đảo trong hầu hết hiệp đầu nhưng Ý đã không thể ghi thêm try nào trước khi nghỉ giữa giờ, tuy nhiên David Bortolussi đã sút ba quả phạt đền kéo dài thế dẫn đầu của họ.
['Chiếm thế áp đảo trong hầu hết hiệp đầu nhưng Ý đã không thể ghi thêm try nào trước khi nghỉ giữa giờ, tuy nhiên David Bortolussi đã sút ba quả phạt đền kéo dài thế dẫn đầu của họ.']
--------------------------------------------------
Bồ Đào Nha chưa bao giờ từ bỏ và David Penalva đã ghi một try ở phút thứ 33, điểm duy nhất của họ trong trận đấu.
['B

In [56]:
len(vi_doc)

56

In [96]:
text = '''Theo phóng viên TTXVN tại New Delhi, ngày 30/12, Thương vụ Việt Nam tại Ấn Độ phối hợp với Phòng Thương mại và Công nghiệp các nhà nhập khẩu Ấn Độ (IICCI) tổ chức hội nghị giao thương trực tuyến với chủ đề “Khám phá cơ hội đầu tư và kinh doanh Việt Nam-Ấn Độ trong lĩnh vực dệt may”. Tham dự buổi giao thương về phía Việt Nam có Tiến sĩ Đỗ Thanh Hải, Tham tán, Đại biện lâm thời Việt Nam tại Ấn Độ, ông Bùi Trung Thướng, Tham tán, Trưởng Cơ quan Thương vụ Việt Nam tại Ấn Độ, ông Bùi Trọng Thoan, Phó Tổng giám đốc Trung tâm thông tin và tư vấn đầu tư, Hiệp hội doanh nghiệp đầu tư nước ngoài thuộc Bộ Kế hoạch và Đầu tư, bà Phạm Minh Hương, nguyên giám đốc điều hành Tập đoàn dệt may Việt Nam. Về phía Ấn Độ có ông A.K Saxena, Chủ tịch IICCI, ông A. Sakthivel, Chủ tịch hội đồng xúc tiến xuất khẩu dệt may - Bộ Dệt May Ấn Độ, ông Manmonhan Agarawal, Phó Chủ tịch Hiệp hội công nghiệp Ấn Độ và khoảng 100 doanh nghiệp lớn thuộc lĩnh vực dệt may của 2 nước. Trong ảnh: Hội thảo giao thương trực tuyến "Khám phá cơ hội đầu tư và kinh doanh Việt Nam-Ấn Độ trong lĩnh vực dệt may”. Ảnh: Huy Lê - P/v TTXVN tại Ấn Độ. Theo ông Sakthivel, ngành dệt may của Ấn Độ đóng góp hơn 2% vào GDP của nước này và tạo việc làm cho hơn 45 triệu người, đóng góp 15% vào thu nhập xuất khẩu trong năm tài chính 2018-1019. Thị trường hàng dệt may của Ấn Độ ước tính đạt 100 tỷ USD trong tài khóa 2018-2019. Chính phủ Ấn Độ đang áp dụng nhiều biện pháp để thúc đẩy ngành dệt may, trong đó cho phép đầu tư 100% vốn đầu tư trực tiếp nước ngoài (FDI) vào lĩnh vực này theo lộ trình tự động. Đặc biệt, tháng 11/2020, Chính phủ Ấn Độ đã thông qua chương trình khuyến khích liên kết sản xuất đối với lĩnh vực dệt may. Đề án này cung cấp các ưu đãi cho sản xuất và xuất khẩu các sản phẩm dệt, đặc biệt là những sản phẩm làm từ sợi nhân tạo (MMF). Về phần mình, ông Bùi Trọng Thoan, Phó Tổng giám đốc Trung tâm thông tin và tư vấn đầu tư, Hiệp hội doanh nghiệp đầu tư nước ngoài, khẳng định Việt Nam là một điểm sáng trong bức tranh kinh tế toàn cầu ảm đạm do dịch bệnh COVID-19 hoành hành. Việt Nam là một trong số ít quốc gia ghi nhận mức tăng trưởng dương năm 2020 với tốc độ đạt 2,91%, tổng kim ngạch xuất nhập khẩu 543,9 tỷ USD, thặng dư thương mại 19,1 tỷ USD, dự trữ ngoại hối khoảng 100 tỷ USD. Về vốn FDI, Việt Nam có chính sách thu hút đầu tư nước ngoài rộng mở, năm 2020 thu hút đầu tư nước ngoài tuy có giảm khoảng 13% so với năm 2019 nhưng dự kiến sẽ tăng mạnh trong các năm tiếp theo do sức hút của các hiệp định thương mại tự do Việt Nam mới kí kết, trong đó có Hiệp định đối tác toàn diện và tiến bộ xuyên Thái Bình Dương (CPTPP), Hiệp định thương mại tự do Việt Nam-Liên minh châu Âu (EVFTA), Hiệp định đối tác kinh tế toàn diện khu vực (RCEP). Đối với lĩnh vực dệt may, ông Thoan cho rằng sức hút FDI sẽ gia tăng do EU, Trung quốc, Nhật Bản – là những thị trường tiêu thụ sản phẩm dệt may lớn -  đều có FTA với Việt Nam. Theo thông tin từ hội thảo, Việt Nam và Ấn Độ có nhiều tiềm năng thúc đẩy hợp tác trong lĩnh vực dệt may. Từ tháng 1-11/2020, Việt Nam nhập khẩu khoảng 1,2 tỷ USD bông và sợi polieste, trong đó 62% đến từ Mỹ, 18% từ Brazil, và chỉ 7% từ Ấn Độ. Đối với vải, Việt Nam nhập khẩu khoảng 10,7 tỷ, trong đó khoảng 60% từ Trung Quốc, còn lại là từ Hàn Quốc, Đài Loan, Nhật Bản và một số nước, Ấn Độ chỉ chiếm khoảng 1%. Trong khi đó, Ấn Độ sở hữu nền công nghiệp dệt may lâu đời, với thế mạnh dựa trên hàng loạt các loại sợi tự nhiên như bông, đay, tơ tằm, len cho đến các loại sợi tổng hợp tổng hợp như polyster, nylon, có nền công nghệ phát triển. Lợi thế này sẽ là sự bổ sung quý giá cho Việt Nam vốn phụ thuộc nhiều vào nguồn nguyên liệu nhập khẩu cho ngành dệt may cũng như mở ra triển vọng thúc đẩy hợp tác với Việt Nam trong đào tạo lực lượng lao động dệt may, ứng dụng khoa học công nghệ, thiết kế…/.
'''

In [99]:
sent_tokenize(text)

['Theo phóng viên TTXVN tại New Delhi, ngày 30/12, Thương vụ Việt Nam tại Ấn Độ phối hợp với Phòng Thương mại và Công nghiệp các nhà nhập khẩu Ấn Độ (IICCI) tổ chức hội nghị giao thương trực tuyến với chủ đề “Khám phá cơ hội đầu tư và kinh doanh Việt Nam-Ấn Độ trong lĩnh vực dệt may”.',
 'Tham dự buổi giao thương về phía Việt Nam có Tiến sĩ Đỗ Thanh Hải, Tham tán, Đại biện lâm thời Việt Nam tại Ấn Độ, ông Bùi Trung Thướng, Tham tán, Trưởng Cơ quan Thương vụ Việt Nam tại Ấn Độ, ông Bùi Trọng Thoan, Phó Tổng giám đốc Trung tâm thông tin và tư vấn đầu tư, Hiệp hội doanh nghiệp đầu tư nước ngoài thuộc Bộ Kế hoạch và Đầu tư, bà Phạm Minh Hương, nguyên giám đốc điều hành Tập đoàn dệt may Việt Nam.',
 'Về phía Ấn Độ có ông A.K Saxena, Chủ tịch IICCI, ông A. Sakthivel, Chủ tịch hội đồng xúc tiến xuất khẩu dệt may - Bộ Dệt May Ấn Độ, ông Manmonhan Agarawal, Phó Chủ tịch Hiệp hội công nghiệp Ấn Độ và khoảng 100 doanh nghiệp lớn thuộc lĩnh vực dệt may của 2 nước.',
 'Trong ảnh: Hội thảo giao thươ

In [70]:
vi_doc[0]

'Theo phóng viên TTXVN tại New Delhi, ngày 30/12, Thương vụ Việt Nam tại Ấn Độ phối hợp với Phòng Thương mại và Công nghiệp các nhà nhập khẩu Ấn Độ (IICCI) tổ chức hội nghị giao thương trực tuyến với chủ đề “Khám phá cơ hội đầu tư và kinh doanh Việt Nam-Ấn Độ trong lĩnh vực dệt may”. Tham dự buổi giao thương về phía Việt Nam có Tiến sĩ Đỗ Thanh Hải, Tham tán, Đại biện lâm thời Việt Nam tại Ấn Độ, ông Bùi Trung Thướng, Tham tán, Trưởng Cơ quan Thương vụ Việt Nam tại Ấn Độ, ông Bùi Trọng Thoan, Phó Tổng giám đốc Trung tâm thông tin và tư vấn đầu tư, Hiệp hội doanh nghiệp đầu tư nước ngoài thuộc Bộ Kế hoạch và Đầu tư, bà Phạm Minh Hương, nguyên giám đốc điều hành Tập đoàn dệt may Việt Nam. Về phía Ấn Độ có ông A.K Saxena, Chủ tịch IICCI, ông A. Sakthivel, Chủ tịch hội đồng xúc tiến xuất khẩu dệt may - Bộ Dệt May Ấn Độ, ông Manmonhan Agarawal, Phó Chủ tịch Hiệp hội công nghiệp Ấn Độ và khoảng 100 doanh nghiệp lớn thuộc lĩnh vực dệt may của 2 nước. Trong ảnh: Hội thảo giao thương trực tuyến

In [129]:
vi_doc_preprocessed = preprocess_text(vi_doc, 'vi')

In [46]:
km_doc_preprocessed = preprocess_text(km_doc, 'km')

In [48]:
km_doc_preprocessed

['ចាប់តាំងពីចូលរួមក្នុងវេទិកា apec ប្រទេសវៀតណាមគឺជាសមាជិកសកម្មក្នុងចំណោមបណ្តាសមាជិកដែលបានលើកឡើងយ៉ាងសកម្មនិងចូលរួមប្រកបដោយជោគជ័យនៅក្នុងគំនិតផ្តួចផ្តើមជាង ៨០ លើវិស័យជាច្រើន រួមមាន៖',
 'ពាណិជ្ជកម្ម ការវិនិយោគ កិច្ចសហប្រតិបត្តិការខាងបច្ចេកទេសនិងសុខភាព…ចូលរួមចំណែកក្នុងការដោះស្រាយបញ្ហាប្រឈមដែលកំពុងកើតឡើងសំរាប់ apec តាមរយៈការផ្តួចផ្តើមគំនិតដើម្បីកាត់បន្ថយគំលាតនៃការអភិវឌ្ឍន៍ ដោះស្រាយនឹងការគំរាមកំហែងនៃបញ្ហាភេរវកម្ម គ្រោះមហន្តរាយធម្មជាតិ ជំងឺរាតត្បាត ការធ្វើកំណែទម្រង់ apec ប្រកបដោយប្រសិទ្ធភាពនិងខ្លាំងក្លា ដែលស្របនឹងការវិវឌ្ឍថ្មីនៃស្ថានភាពក្នុងតំបន់និងពិភពលោក…',
 'ជាពិសេស នៅឆ្នាំ ២០២០ វៀតណាមបានចូលរួមយ៉ាងសកម្មក្នុងកិច្ចប្រជុំនិងសន្និសីទ apec ចំនួនជិត ១០០ ទាំងដោយផ្ទាល់និងតាមប្រព័ន្ធវេដេអូ ជាពិសេសអញ្ជើញចូលរួមក្នុងកិច្ចប្រជុំ  កិច្ចសន្ទនាថ្នាក់រដ្ឋមន្ត្រីចំនួន ៩ រួមមាន៖',
 'កិច្ចប្រជុំរដ្ឋមន្ត្រីទេសចរណ៍កិច្ចប្រជុំរដ្ឋមន្ត្រីក្រសួងពាណិជ្ជកម្ម កិច្ចប្រជុំកំពូលស្តីពីសុខាភិបាលនិងសេដ្ឋកិច្ច កិច្ចសន្ទនាគោលនយោបាយជាន់ខ្ពស់ស្តីពីសន្តិសុខស្បៀងអាហារ កិច្ចប្រជុំ ថ្នាក់រដ្ឋមន្ត្រីស្តីពីការកែទម្រង់រចនាសម្ព័ន្ធ កិច

In [53]:
vi_doc_segmented = [segment_sentence(sen) for sen in vi_doc_preprocessed]

2021-01-05 00:09:05,168 - DEBUG    - urllib3.connectionpool    - Starting new HTTP connection (1): 127.0.0.1:52582
2021-01-05 00:09:05,174 - DEBUG    - urllib3.connectionpool    - http://127.0.0.1:52582 "POST /handle HTTP/1.1" 200 None
2021-01-05 00:09:05,181 - DEBUG    - urllib3.connectionpool    - Starting new HTTP connection (1): 127.0.0.1:52582
2021-01-05 00:09:05,186 - DEBUG    - urllib3.connectionpool    - http://127.0.0.1:52582 "POST /handle HTTP/1.1" 200 None
2021-01-05 00:09:05,196 - DEBUG    - urllib3.connectionpool    - Starting new HTTP connection (1): 127.0.0.1:52582
2021-01-05 00:09:05,202 - DEBUG    - urllib3.connectionpool    - http://127.0.0.1:52582 "POST /handle HTTP/1.1" 200 None
2021-01-05 00:09:05,209 - DEBUG    - urllib3.connectionpool    - Starting new HTTP connection (1): 127.0.0.1:52582
2021-01-05 00:09:05,215 - DEBUG    - urllib3.connectionpool    - http://127.0.0.1:52582 "POST /handle HTTP/1.1" 200 None
2021-01-05 00:09:05,221 - DEBUG    - urllib3.connectionp

In [54]:
vi_doc_segmented

['ý đã đánh_bại bồ_đào_nha với tỉ_số 31-5 ở bảng c giải vô_địch rugby thế_giới 2007 tại parc des princes pari pháp',
 'andrea maisi đã mở tỉ_số cho ý ở phút thứ tư với một quả try',
 'chiếm thế áp_đảo trong hầu_hết hiệp đầu nhưng ý đã không_thể ghi thêm try nào trước khi nghỉ giữa giờ tuy_nhiên david bortolussi đã sút ba quả phạt_đền kéo_dài thế dẫn_đầu của họ',
 'bồ_đào_nha chưa bao_giờ từ_bỏ và david penalva đã ghi một try ở phút thứ 33 điểm duy_nhất của họ trong trận đấu',
 'ý đã dẫn 16-5 ở hiệp đầu nhưng ngang sức với bồ_đào_nha ở hầu_hết thời_gian của hiệp hai',
 'tuy_nhiên bortolussi đã ghi cú phạt_đền lần thứ tư của trận đấu tiếp đó là những quả try của mauro bergammasco và quả try thứ hai của andrea masi chốt lại chiến_thắng cho ý',
 'hiện_tại đứng ở vị_trí thứ ba trong bảng c với tám điểm ý phải đối_mặt với một trận đấu gay_go trước scốt-len đang ở vị_trí thứ hai vào ngày 29 tháng chín',
 'niu-di-lân dẫn_đầu bảng với mười điểm hơn scốt-len ở hiệu_số bàn thắng',
 'bồ_đào_nha ở 

In [57]:
encoded_input = tokenizer(vi_doc_segmented[2])['input_ids']

In [59]:
for word in encoded_input: 
    print(tokenizer.decode([word]))

<s>
chiếm
thế
áp_đảo
trong
hầu_hết
hiệp
đầu
nhưng
ý
đã
không_thể
ghi
thêm
try
nào
trước
khi
nghỉ
giữa
giờ
tuy_nhiên
dav@@
id
bor@@
to@@
lus@@
si
đã
sút
ba
quả
phạt_đền
kéo_dài
thế
dẫn_đầu
của
họ
</s>


In [58]:
tokenizer.decode([])

[0,
 724,
 570,
 8075,
 12,
 1399,
 2506,
 127,
 51,
 2198,
 14,
 279,
 701,
 143,
 17168,
 142,
 71,
 26,
 915,
 147,
 232,
 1397,
 51154,
 5916,
 10502,
 2225,
 25029,
 7931,
 14,
 1971,
 346,
 645,
 10538,
 832,
 570,
 2237,
 7,
 86,
 2]